# This notebook contains code to train and predict ensemble classifier. It also contains the code to combine task3 ner results. For task3, the ner prediction file and QnA file must be present

Link to code containing Ensemble NER:
https://colab.research.google.com/drive/1CCFJ_0Py2Sz4GjWnP7rrlUaMdry0DPYP?authuser=1#scrollTo=gW6kwZCyS3G_

In [0]:
%%bash
pip install -q transformers

In [0]:
%%bash
pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=74273566962c883c7b8513ec80c714fa1baefd857595537f7d9a9c11d543ce85
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# !cp /content/drive/My\ Drive/biobert_large_v1.1_pubmed.tar.gz ./

In [0]:
# !tar xvzf biobert_large_v1.1_pubmed.tar.gz

In [0]:
# !transformers-cli convert --model_type bert \
#   --tf_checkpoint biobert_large/bio_bert_large_1000k.ckpt \
#   --config biobert_large/bert_config_bio_58k_large.json \
#   --pytorch_dump_output biobert_large/pytorch_model.bin

In [0]:
# !cp biobert_large/bert_config_bio_58k_large.json biobert_large/config.json
# !cp biobert_large/vocab_cased_pubmed_pmc_30k.txt biobert_large/vocab.txt

In [0]:
# !ls -lr biobert_large

In [0]:
# !cp -r ./biobert_large /content/drive/My\ Drive/

In [0]:
# from transformers import BertModel
# model = BertModel.from_pretrained('biobert_large')

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path 

import os

import torch
import torch.optim as optim
import copy
import random 

# transformers
import transformers
from transformers import BertModel, BertTokenizer, BertConfig
from transformers import RobertaModel, RobertaTokenizer, RobertaConfig

In [4]:
train = pd.read_csv('train_v4.csv')
dev = pd.read_csv('dev_v4.csv')
# test = pd.read_csv('test_v3.csv')
print(train.shape, dev.shape)

(25095, 2) (5134, 2)


In [5]:
test2 = pd.read_csv('tweet_mapping.csv')
print(test2.shape)
test2.head(2)

(5134, 2)


,cleaned_tweet,tweet_id
0,i do not fucking need humira,343909778008973312
1,"my retake is next friday , if i bloody fail ag...",352823276889837570


In [0]:
x_train = list(train['tweet'])
y_train = list(train['class'])

x_dev = list(dev['cleaned_tweet'])
y_dev = list(dev['class'])

# x_test = list(test['cleaned_tweet'])
# y_test = list(test['class'])

In [0]:
# x_test2 = list(test2['cleaned_tweet'])

In [0]:
MAX_LEN = 128
bs = 16
model_name = 'roberta-large'#'biobert_large' #roberta-large

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print(device)

cuda


In [0]:
if model_name == 'biobert_large':
  tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)
else:
  tokenizer = RobertaTokenizer.from_pretrained(model_name, do_lower_case=True)
print(tokenizer.cls_token, tokenizer.cls_token_id)
print(tokenizer.pad_token, tokenizer.pad_token_id)
print(tokenizer.sep_token, tokenizer.sep_token_id)
print(tokenizer.unk_token, tokenizer.unk_token_id)


<s> 0
<pad> 1
</s> 2
<unk> 3


In [0]:
def transformer_preprocess(data, tokenizer):
  input_ids = []
  attention_masks = []
  input_types = []
  for i in data:
    tokenizer_op = tokenizer.encode_plus(i, 
                          max_length=MAX_LEN,
                          pad_to_max_length = True,
                          # return_tensors = 'pt',
                          return_token_type_ids = True,
                          return_attention_mask = True)
    input_ids.append(tokenizer_op['input_ids'])
    attention_masks.append(tokenizer_op['attention_mask'])
    input_types.append(tokenizer_op['token_type_ids'])
  return torch.tensor(input_ids), torch.tensor(attention_masks), torch.tensor(input_types)
    

In [0]:
train_input_ids, train_attention_masks, train_input_types = transformer_preprocess(x_train, tokenizer)
train_input_ids.shape, train_attention_masks.shape, train_input_types.shape

(torch.Size([25095, 128]), torch.Size([25095, 128]), torch.Size([25095, 128]))

In [0]:
dev_input_ids, dev_attention_masks, dev_input_types = transformer_preprocess(x_dev, tokenizer)
dev_input_ids.shape, dev_attention_masks.shape, dev_input_types.shape

(torch.Size([5134, 128]), torch.Size([5134, 128]), torch.Size([5134, 128]))

In [0]:
tr_class = torch.tensor(y_train)
val_class = torch.tensor(y_dev)

In [0]:
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

train_data = TensorDataset(train_input_ids, train_attention_masks, 
                           train_input_types, tr_class)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(dev_input_ids, dev_attention_masks, 
                           dev_input_types, val_class)
valid_sampler = RandomSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [0]:
test_sampler = SequentialSampler(valid_data)
test_dataloader = DataLoader(valid_data, sampler=test_sampler, batch_size=bs)

In [0]:
# test2_input_ids, test2_attention_masks, test2_input_types = transformer_preprocess(x_test2, tokenizer)
# test2_input_ids.shape, test2_attention_masks.shape, test2_input_types.shape

In [0]:
if model_name == 'biobert_large':
  config = BertConfig.from_pretrained(model_name)
else:
  config = RobertaConfig.from_pretrained(model_name)
config.output_hidden_states = True

In [0]:
a = [torch.rand(16,10,256),torch.rand(16,10,256),torch.rand(16,10,256)]
torch.stack(a).shape

torch.Size([3, 16, 10, 256])

In [0]:
torch.cat(a, dim=-1).shape

torch.Size([16, 10, 768])

In [0]:
from torch.nn.utils.rnn import pad_sequence

class SequenceClassifier(torch.nn.Module):
  def __init__(self, transformer_model, config, n_layers=4):
    super().__init__()
    self.transformer = transformer_model
    self.out = torch.nn.Linear(config.hidden_size*2, 2)
    self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
    self.n_layers = n_layers
  
  def forward(self, input_ids, attention_mask, segment_id, classification_labels=None):
    
    #Batch max length
    max_length = (attention_mask != 0).max(0)[0].nonzero()[-1].item()+1

    if max_length < input_ids.shape[1]:
      input_ids = input_ids[:, :max_length]
      attention_mask = attention_mask[:, :max_length]

    segment_id = torch.zeros_like(attention_mask)
    hidden = self.transformer(input_ids = input_ids,attention_mask = attention_mask, 
                              token_type_ids = segment_id)
    #Aggregating all the hidden states
    # token_hidden = hidden[2]
    token_hidden = hidden[2][-self.n_layers:]
    # token_hidden = torch.cat(token_hidden, dim=-1)
    # token_hidden = torch.sum(token_hidden, dim=1)
    token_hidden = torch.mean(torch.sum(torch.stack(token_hidden), dim=0), 
                              dim=1)

    classifier_hidden = hidden[1]
    hidden_cat = token_hidden
    hidden_cat = torch.cat([token_hidden, classifier_hidden], dim=1)

    classification_logits = self.out(self.dropout(hidden_cat))
    outputs = [classification_logits]
    if classification_labels is not None:
      loss_fct_classification = torch.nn.CrossEntropyLoss()

      loss_classification = loss_fct_classification(classification_logits.view(-1, 2), 
                                                      classification_labels.view(-1))

      outputs += [loss_classification]
    return outputs

In [0]:
if model_name == 'biobert_large':
  transformer_model = BertModel.from_pretrained(model_name, config=config)
else:
  transformer_model = RobertaModel.from_pretrained(model_name, config=config)

In [0]:
# for name,param in transformer_model.named_parameters():
#   if 'embedding' in name:
#     param.requires_grad = False

In [0]:
n_layers=4
model = SequenceClassifier(transformer_model, config, n_layers)


In [0]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
model.cuda();

In [0]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=2e-5)
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=2e-5, max_lr=1e-4, 
#                                               cycle_momentum=False)

In [0]:
# from seqeval.metrics import f1_score

def flat_accuracy_classification(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix

epochs = 4
max_grad_norm = 1.0
model.zero_grad()
best_f1 = 0
for _ in tqdm(range(epochs)):
  # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for batch in tqdm(train_dataloader):

      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_segment, b_classes = batch
      
      outputs = model(b_input_ids, b_input_mask, b_segment, b_classes)
      
      loss = outputs[1]
      
      loss.backward()

      tr_loss += loss.item()
      nb_tr_examples += b_input_ids.size(0)
      nb_tr_steps += 1

      torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), 
                                     max_norm=max_grad_norm)

      optimizer.step()
      # scheduler.step()
      model.zero_grad()

    print("Train loss: {}".format(tr_loss/nb_tr_steps))

    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy_class = 0, 0
    
    nb_eval_steps, nb_eval_examples = 0, 0
    class_preds ,class_true_labels = [], []
    

    for batch in valid_dataloader:
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_segment, b_classes = batch

      with torch.no_grad():
        outputs = model(b_input_ids, b_input_mask, b_segment, b_classes)
        eval_loss = classification_loss = outputs[1]

      classification_logits = outputs[0].detach().cpu().numpy()
      class_preds.extend(list(np.argmax(classification_logits,axis=1)))
      
      class_label_ids = b_classes.to('cpu').numpy()
      
      class_true_labels.append(class_label_ids)
      
      tmp_eval_accuracy_class = flat_accuracy_classification(classification_logits, 
                                              class_label_ids)
      eval_loss += eval_loss.mean().item()

      eval_accuracy_class += tmp_eval_accuracy_class
      nb_eval_examples += b_input_ids.size(0)
      nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy Classifier: {}".format(eval_accuracy_class/nb_eval_steps))
    valid_tags = [l_i for l in class_true_labels for l_i in l ]
    pred_tags = [p for p in class_preds]
    print("Val F1 ",f1_score(valid_tags, pred_tags))
    print("Val Precision ",precision_score(valid_tags, pred_tags))
    print("Val Recall ",recall_score(valid_tags, pred_tags))
    print("Val Accuracy ",accuracy_score(valid_tags, pred_tags))
    print("Confusion matrix \n", confusion_matrix(valid_tags, pred_tags))
    if f1_score(valid_tags, pred_tags) >= best_f1:
      print("Saving this model")
      torch.save(model.state_dict(), './best_model_state_dict_roberta_aggregate_all_hidden')
      # torch.save(model, './best_model')
      print("Copying to G drive")
      !cp ./best_model_state_dict_roberta_aggregate_all_hidden drive/My\ Drive/
      # !cp ./best_model drive/My\ Drive/
      print("Copied to G drive")
      best_f1 = f1_score(valid_tags, pred_tags)



  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1569 [00:00<?, ?it/s]

  0%|          | 1/1569 [00:00<14:18,  1.83it/s]

  0%|          | 2/1569 [00:00<12:20,  2.12it/s]

  0%|          | 3/1569 [00:01<10:46,  2.42it/s]

  0%|          | 4/1569 [00:01<09:57,  2.62it/s]

  0%|          | 5/1569 [00:01<09:15,  2.82it/s]

  0%|          | 6/1569 [00:02<08:47,  2.96it/s]

  0%|          | 7/1569 [00:02<08:23,  3.11it/s]

  1%|          | 8/1569 [00:02<08:14,  3.16it/s]

  1%|          | 9/1569 [00:02<08:05,  3.21it/s]

  1%|          | 10/1569 [00:03<08:11,  3.17it/s]

  1%|          | 11/1569 [00:03<08:02,  3.23it/s]

  1%|          | 12/1569 [00:03<07:55,  3.28it/s]

  1%|          | 13/1569 [00:04<07:46,  3.34it/s]

  1%|          | 14/1569 [00:04<07:33,  3.43it/s]

  1%|          | 15/1569 [00:04<07:43,  3.35it/s]

  1%|          | 16/1569 [00:04<07:38,  3.39it/s]

  1%|          | 17/1569 [00:05<07:34,  3.41it/s]

  1%|          | 18/1569 [00:05<07:37,  3.39it/s]

  1%|    

Train loss: 0.3074442775785885
Validation loss: 0.000817202846519649
Validation Accuracy Classifier: 0.9135235870048954
Val F1  0.612565445026178
Val Precision  0.5223214285714286
Val Recall  0.740506329113924
Val Accuracy  0.913517724970783
Confusion matrix 
 [[4339  321]
 [ 123  351]]
Saving this model
Copying to G drive



 25%|██▌       | 1/4 [09:28<28:25, 568.66s/it]

  0%|          | 0/1569 [00:00<?, ?it/s]

Copied to G drive




  0%|          | 1/1569 [00:00<07:55,  3.30it/s]

  0%|          | 2/1569 [00:00<07:52,  3.32it/s]

  0%|          | 3/1569 [00:00<08:22,  3.11it/s]

  0%|          | 4/1569 [00:01<08:22,  3.11it/s]

  0%|          | 5/1569 [00:01<08:19,  3.13it/s]

  0%|          | 6/1569 [00:01<08:23,  3.10it/s]

  0%|          | 7/1569 [00:02<08:22,  3.11it/s]

  1%|          | 8/1569 [00:02<08:23,  3.10it/s]

  1%|          | 9/1569 [00:02<08:07,  3.20it/s]

  1%|          | 10/1569 [00:03<07:59,  3.25it/s]

  1%|          | 11/1569 [00:03<07:49,  3.32it/s]

  1%|          | 12/1569 [00:03<07:43,  3.36it/s]

  1%|          | 13/1569 [00:04<07:59,  3.25it/s]

  1%|          | 14/1569 [00:04<07:44,  3.35it/s]

  1%|          | 15/1569 [00:04<07:43,  3.35it/s]

  1%|          | 16/1569 [00:04<07:49,  3.31it/s]

  1%|          | 17/1569 [00:05<07:56,  3.26it/s]

  1%|          | 18/1569 [00:05<07:57,  3.25it/s]

  1%|          | 19/1569 [00:05<07:49,  3.30it/s]

  1%|▏         | 20/1569 [00:06<07:42,

...

Copied to G drive




  0%|          | 1/1569 [00:00<08:44,  2.99it/s]

  0%|          | 2/1569 [00:00<08:27,  3.09it/s]

  0%|          | 3/1569 [00:00<08:11,  3.18it/s]

  0%|          | 4/1569 [00:01<08:04,  3.23it/s]

  0%|          | 5/1569 [00:01<08:15,  3.15it/s]

  0%|          | 6/1569 [00:01<08:01,  3.25it/s]

  0%|          | 7/1569 [00:02<08:10,  3.19it/s]

  1%|          | 8/1569 [00:02<07:59,  3.25it/s]

  1%|          | 9/1569 [00:02<07:55,  3.28it/s]

  1%|          | 10/1569 [00:03<07:43,  3.36it/s]

  1%|          | 11/1569 [00:03<07:57,  3.26it/s]

  1%|          | 12/1569 [00:03<07:46,  3.34it/s]

  1%|          | 13/1569 [00:03<07:44,  3.35it/s]

  1%|          | 14/1569 [00:04<07:50,  3.31it/s]

  1%|          | 15/1569 [00:04<07:42,  3.36it/s]

  1%|          | 16/1569 [00:04<07:52,  3.29it/s]

  1%|          | 17/1569 [00:05<07:42,  3.36it/s]

  1%|          | 18/1569 [00:05<07:37,  3.39it/s]

  1%|          | 19/1569 [00:05<07:36,  3.39it/s]

  1%|▏         | 20/1569 [00:06<07:48,

Train loss: 0.08338637230396345



100%|██████████| 4/4 [34:55<00:00, 523.90s/it]

Validation loss: 2.3867321488069138e-06
Validation Accuracy Classifier: 0.9215342679127726
Val F1  0.6292548298068077
Val Precision  0.5579119086460033
Val Recall  0.7215189873417721
Val Accuracy  0.9215037008180755
Confusion matrix 
 [[4389  271]
 [ 132  342]]


In [0]:
# !cp drive/My\ Drive/best_model ./

In [0]:
model = SequenceClassifier(transformer_model, config, n_layers)
model.load_state_dict(torch.load('./best_model_state_dict_roberta_aggregate_all_hidden'))
model.eval();

In [0]:
for name,param in model.named_parameters():
  param.requires_grad = False
model.cuda();

In [0]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix
eval_loss, eval_accuracy_class = 0, 0
    
nb_eval_steps, nb_eval_examples = 0, 0
class_preds ,class_true_labels = [], []

for batch in test_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_segment, y_true = batch

  with torch.no_grad():
    outputs = model(b_input_ids, b_input_mask, b_segment)
    #eval_loss = classification_loss = outputs[1]

  classification_logits = outputs[0].detach().cpu().numpy()
  class_preds.extend(list(np.argmax(classification_logits,axis=1)))


In [0]:
from sklearn.metrics import f1_score, accuracy_score
print(f1_score(list(dev['class']) ,class_preds))
print(accuracy_score(list(dev['class']) ,class_preds))

0.6384544618215271
0.9234514998052201


In [0]:
test_predictions = pd.concat([test2.reset_index(drop=True), 
                              pd.DataFrame(class_preds, columns = ['pred']),
                              pd.DataFrame(list(dev['class']), columns = ['true'])], axis=1)
print(test_predictions.shape)
test_predictions.head(2)

(5134, 4)


,cleaned_tweet,tweet_id,pred,true
0,i do not fucking need humira,343909778008973312,0,0
1,"my retake is next friday , if i bloody fail ag...",352823276889837570,0,0


In [0]:
final_op = test_predictions[['tweet_id', 'pred']].drop_duplicates()
final_op.columns = ['tweet_id', 'Class']
print(final_op.shape)
final_op.head(2)

(5134, 2)


,tweet_id,Class
0,343909778008973312,0
1,352823276889837570,0


In [0]:
final_op.to_csv('prediction_task2_en_roberta.tsv',sep = '\t', index=False)

In [0]:
test_predictions.to_csv('test_prediction_task2_en_roberta.tsv',sep = '\t', index=False)

# Sklearn BERT

In [9]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .
import os
os.chdir("bert-sklearn")
print(os.listdir())

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 259 (delta 3), reused 3 (delta 0), pack-reused 247
Receiving objects: 100% (259/259), 519.36 KiB | 17.91 MiB/s, done.
Resolving deltas: 100% (125/125), done.
Processing /content/bert-sklearn
  Created wheel for bert-sklearn: filename=bert_sklearn-0.3.1-cp36-none-any.whl size=54234 sha256=c47b55df66beddbd0a01e62d6281d6809826b37a5b0e5a0985bd37fa6da8fc23
  Stored in directory: /root/.cache/pip/wheels/61/95/c6/5790aae8fb377f5ff356dbe58205aab28858595d6bff8197d0
Successfully built bert-sklearn
['LICENSE', 'other_examples', 'glue_examples', 'demo_tuning_hyperparams.ipynb', 'tests', 'demo.ipynb', 'setup.py', '.git', 'Options.md', 'bert_sklearn', 'README.md']


In [0]:
import os
import math
import random
import csv
import sys

import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import statistics as stats

from bert_sklearn import BertClassifier
from bert_sklearn import BertRegressor
from bert_sklearn import BertTokenClassifier
from bert_sklearn import load_model

In [0]:
train = pd.read_csv('../train_v4.csv')
dev = pd.read_csv('../dev_v4.csv')

In [12]:
train.shape, dev.shape

((25095, 2), (5134, 2))

In [13]:
train.head(2)

,tweet,class
0,do you always get side effects with fluoxetine ?,0
1,i am on fluoxetine atm and i have had nothing ...,0


In [0]:
X_train = train['tweet']
y_train = train['class']

# BERT Base

In [0]:
# define model
model = BertClassifier()
model.epochs = 4
model.validation_fraction = 0.1
model.learning_rate = 2e-5
model.max_seq_length = 128
model.gradient_accumulation_steps = 4
print('\n',model,'\n')


Building sklearn text classifier...

 BertClassifier(bert_config_json=None, bert_model='bert-base-uncased',
               bert_vocab=None, do_lower_case=None, epochs=4, eval_batch_size=8,
               fp16=False, from_tf=False, gradient_accumulation_steps=4,
               ignore_label=None, label_list=None, learning_rate=2e-05,
               local_rank=-1, logfile='bert_sklearn.log', loss_scale=0,
               max_seq_length=128, num_mlp_hiddens=500, num_mlp_layers=0,
               random_state=42, restore_file=None, train_batch_size=32,
               use_cuda=True, validation_fraction=0.1, warmup_proportion=0.1) 



In [0]:
model.fit(X_train, y_train)

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 22586, validation data size: 2509





Epoch 1, Train loss: 0.2816, Val loss: 0.1334, Val accy: 95.26%





Epoch 2, Train loss: 0.0635, Val loss: 0.0645, Val accy: 97.73%





Epoch 3, Train loss: 0.0178, Val loss: 0.0785, Val accy: 97.53%





Epoch 4, Train loss: 0.0061, Val loss: 0.1027, Val accy: 97.41%



BertClassifier(bert_config_json=None, bert_model='bert-base-uncased',
               bert_vocab=None, do_lower_case=True, epochs=4, eval_batch_size=8,
               fp16=False, from_tf=False, gradient_accumulation_steps=4,
               ignore_label=None, label_list=array([0, 1]), learning_rate=2e-05,
               local_rank=-1, logfile='bert_sklearn.log', loss_scale=0,
               max_seq_length=128, num_mlp_hiddens=500, num_mlp_layers=0,
               random_state=42, restore_file=None, train_batch_size=32,
               use_cuda=True, validation_fraction=0.1, warmup_proportion=0.1)

In [7]:
test = dev
X_test = dev['cleaned_tweet']
y_test = dev['class']

# make predictions
y_pred = model.predict(X_test)
print("Accuracy: %0.2f%%"%(metrics.accuracy_score(y_pred,y_test) * 100))
print(classification_report(y_test, y_pred, target_names=['negative','positive']))

NameError: ignored

In [0]:
from sklearn.metrics import f1_score, accuracy_score
print(f1_score(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

0.6035751840168243
0.9265679781846513


In [0]:
savefile='./bert-base-uncased-task2-en.bin'
model.save(savefile)

In [0]:
!pwd

/content/bert-sklearn


In [0]:
!cp ./bert-base-uncased-task2-en.bin ../drive/My\ Drive/

# bioBERT v1.1 base

In [15]:
# define model
model = BertClassifier('biobert-v1.1-pubmed-base-cased')
# model.model = 'bert-large-uncased'
model.epochs = 4
model.validation_fraction = 0.1
model.learning_rate = 2e-5
model.max_seq_length = 128
model.gradient_accumulation_steps = 4
model.train_batch_size = 8
print('\n',model,'\n')


Building sklearn text classifier...

 BertClassifier(bert_config_json=None,
               bert_model='biobert-v1.1-pubmed-base-cased', bert_vocab=None,
               do_lower_case=None, epochs=4, eval_batch_size=8, fp16=False,
               from_tf=False, gradient_accumulation_steps=4, ignore_label=None,
               label_list=None, learning_rate=2e-05, local_rank=-1,
               logfile='bert_sklearn.log', loss_scale=0, max_seq_length=128,
               num_mlp_hiddens=500, num_mlp_layers=0, random_state=42,
               restore_file=None, train_batch_size=8, use_cuda=True,
               validation_fraction=0.1, warmup_proportion=0.1) 



In [16]:
model.fit(X_train, y_train)

100%|██████████| 401403346/401403346 [00:10<00:00, 38057600.53B/s]


Loading biobert-v1.1-pubmed-base-cased model...


100%|██████████| 401403346/401403346 [00:10<00:00, 37447409.27B/s]


Defaulting to linear classifier/regressor
Loading Tensorflow checkpoint from  model.ckpt-1000000

train data size: 22586, validation data size: 2509



/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)




Epoch 1, Train loss: 0.2859, Val loss: 0.1344, Val accy: 95.10%





Epoch 2, Train loss: 0.0757, Val loss: 0.1003, Val accy: 96.65%





Epoch 3, Train loss: 0.0193, Val loss: 0.1851, Val accy: 96.61%





Epoch 4, Train loss: 0.0060, Val loss: 0.1363, Val accy: 97.69%



BertClassifier(bert_config_json=None,
               bert_model='biobert-v1.1-pubmed-base-cased', bert_vocab=None,
               do_lower_case=False, epochs=4, eval_batch_size=8, fp16=False,
               from_tf=False, gradient_accumulation_steps=4, ignore_label=None,
               label_list=array([0, 1]), learning_rate=2e-05, local_rank=-1,
               logfile='bert_sklearn.log', loss_scale=0, max_seq_length=128,
               num_mlp_hiddens=500, num_mlp_layers=0, random_state=42,
               restore_file=None, train_batch_size=8, use_cuda=True,
               validation_fraction=0.1, warmup_proportion=0.1)

In [17]:
test = dev
X_test = dev['cleaned_tweet']
y_test = dev['class']

# make predictions
y_pred = model.predict(X_test)
print("Accuracy: %0.2f%%"%(metrics.accuracy_score(y_pred,y_test) * 100))
print(classification_report(y_test, y_pred, target_names=['negative','positive']))


Accuracy: 91.98%
              precision    recall  f1-score   support

    negative       0.96      0.95      0.96      4660
    positive       0.56      0.59      0.57       474

    accuracy                           0.92      5134
   macro avg       0.76      0.77      0.77      5134
weighted avg       0.92      0.92      0.92      5134



In [18]:
from sklearn.metrics import f1_score, accuracy_score
print(f1_score(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

0.5743801652892563
0.9197506817296455


In [0]:
savefile='./biobert-v1.1-pubmed-base-cased-en.bin'
model.save(savefile)

In [0]:
!cp ./biobert-v1.1-pubmed-base-cased-en.bin ../drive/My\ Drive/

# sciBERT scivocab base uncased

In [21]:
# define model
model = BertClassifier('scibert-scivocab-uncased')
# model.model = 'bert-large-uncased'
model.epochs = 4
model.validation_fraction = 0.1
model.learning_rate = 2e-5
model.max_seq_length = 128
model.gradient_accumulation_steps = 4
model.train_batch_size = 8
print('\n',model,'\n')


Building sklearn text classifier...

 BertClassifier(bert_config_json=None, bert_model='scibert-scivocab-uncased',
               bert_vocab=None, do_lower_case=None, epochs=4, eval_batch_size=8,
               fp16=False, from_tf=False, gradient_accumulation_steps=4,
               ignore_label=None, label_list=None, learning_rate=2e-05,
               local_rank=-1, logfile='bert_sklearn.log', loss_scale=0,
               max_seq_length=128, num_mlp_hiddens=500, num_mlp_layers=0,
               random_state=42, restore_file=None, train_batch_size=8,
               use_cuda=True, validation_fraction=0.1, warmup_proportion=0.1) 



In [22]:
model.fit(X_train, y_train)

100%|██████████| 410593280/410593280 [00:08<00:00, 47309553.23B/s]


Loading scibert-scivocab-uncased model...


100%|██████████| 410593280/410593280 [00:06<00:00, 65618731.24B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 22586, validation data size: 2509





Epoch 1, Train loss: 0.3425, Val loss: 0.2304, Val accy: 92.23%





Epoch 2, Train loss: 0.1875, Val loss: 0.1515, Val accy: 94.82%





Epoch 3, Train loss: 0.0927, Val loss: 0.1461, Val accy: 96.17%





Epoch 4, Train loss: 0.0377, Val loss: 0.1212, Val accy: 97.21%



BertClassifier(bert_config_json=None, bert_model='scibert-scivocab-uncased',
               bert_vocab=None, do_lower_case=True, epochs=4, eval_batch_size=8,
               fp16=False, from_tf=False, gradient_accumulation_steps=4,
               ignore_label=None, label_list=array([0, 1]), learning_rate=2e-05,
               local_rank=-1, logfile='bert_sklearn.log', loss_scale=0,
               max_seq_length=128, num_mlp_hiddens=500, num_mlp_layers=0,
               random_state=42, restore_file=None, train_batch_size=8,
               use_cuda=True, validation_fraction=0.1, warmup_proportion=0.1)

In [23]:
test = dev
X_test = dev['cleaned_tweet']
y_test = dev['class']

# make predictions
y_pred = model.predict(X_test)
print("Accuracy: %0.2f%%"%(metrics.accuracy_score(y_pred,y_test) * 100))
print(classification_report(y_test, y_pred, target_names=['negative','positive']))


Accuracy: 92.19%
              precision    recall  f1-score   support

    negative       0.96      0.96      0.96      4660
    positive       0.57      0.59      0.58       474

    accuracy                           0.92      5134
   macro avg       0.77      0.77      0.77      5134
weighted avg       0.92      0.92      0.92      5134



In [24]:
from sklearn.metrics import f1_score, accuracy_score
print(f1_score(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

0.582726326742976
0.9218932606155045


In [0]:
savefile='./scibert-scivocab-uncased-en.bin'
model.save(savefile)

In [0]:
!cp ./scibert-scivocab-uncased-en.bin ../drive/My\ Drive/

# Ensemble prediction

In [0]:
dev = pd.read_csv('../dev_v4.csv')
test = dev
X_test = dev['cleaned_tweet']
y_test = dev['class']

In [0]:
val_class = torch.tensor(list(y_test))

In [0]:
MAX_LEN = 128
bs = 32

In [0]:
# !cp ../drive/My\ Drive/scibert-scivocab-uncased-en.bin ./
# !cp ../drive/My\ Drive/biobert-v1.1-pubmed-base-cased-en.bin ./
# !cp ../drive/My\ Drive/bert-base-uncased-task2-en.bin ./
# !cp ../drive/My\ Drive/best_model_state_dict_roberta_aggregate_all_hidden ./

## Loading roberta pretrained

In [0]:
from torch.nn.utils.rnn import pad_sequence

class SequenceClassifier(torch.nn.Module):
  def __init__(self, transformer_model, config, n_layers=4):
    super().__init__()
    self.transformer = transformer_model
    self.out = torch.nn.Linear(config.hidden_size*2, 2)
    self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
    self.n_layers = n_layers
  
  def forward(self, input_ids, attention_mask, segment_id, classification_labels=None):
    
    #Batch max length
    max_length = (attention_mask != 0).max(0)[0].nonzero()[-1].item()+1

    if max_length < input_ids.shape[1]:
      input_ids = input_ids[:, :max_length]
      attention_mask = attention_mask[:, :max_length]

    segment_id = torch.zeros_like(attention_mask)
    hidden = self.transformer(input_ids = input_ids,attention_mask = attention_mask, 
                              token_type_ids = segment_id)
    #Aggregating all the hidden states
    # token_hidden = hidden[2]
    token_hidden = hidden[2][-self.n_layers:]
    # token_hidden = torch.cat(token_hidden, dim=-1)
    # token_hidden = torch.sum(token_hidden, dim=1)
    token_hidden = torch.mean(torch.sum(torch.stack(token_hidden), dim=0), 
                              dim=1)

    classifier_hidden = hidden[1]
    hidden_cat = token_hidden
    hidden_cat = torch.cat([token_hidden, classifier_hidden], dim=1)

    classification_logits = self.out(self.dropout(hidden_cat))
    outputs = [classification_logits]
    if classification_labels is not None:
      loss_fct_classification = torch.nn.CrossEntropyLoss()

      loss_classification = loss_fct_classification(classification_logits.view(-1, 2), 
                                                      classification_labels.view(-1))

      outputs += [loss_classification]
    return outputs

In [0]:
model_name = 'roberta-large'
config = RobertaConfig.from_pretrained(model_name)
config.output_hidden_states = True
transformer_model = RobertaModel.from_pretrained(model_name, config=config)
n_layers=4
model = SequenceClassifier(transformer_model, config, n_layers)
model.load_state_dict(torch.load('./best_model_state_dict_roberta_aggregate_all_hidden'))

In [43]:
tokenizer = RobertaTokenizer.from_pretrained(model_name)

In [49]:
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

def transformer_preprocess(data, tokenizer):
  input_ids = []
  attention_masks = []
  input_types = []
  for i in data:
    tokenizer_op = tokenizer.encode_plus(i, 
                          max_length=MAX_LEN,
                          pad_to_max_length = True,
                          # return_tensors = 'pt',
                          return_token_type_ids = True,
                          return_attention_mask = True)
    input_ids.append(tokenizer_op['input_ids'])
    attention_masks.append(tokenizer_op['attention_mask'])
    input_types.append(tokenizer_op['token_type_ids'])
  return torch.tensor(input_ids), torch.tensor(attention_masks), torch.tensor(input_types)

dev_input_ids, dev_attention_masks, dev_input_types = transformer_preprocess(list(X_test), tokenizer)
print(dev_input_ids.shape, dev_attention_masks.shape, dev_input_types.shape)

valid_data = TensorDataset(dev_input_ids, dev_attention_masks, 
                           dev_input_types, val_class)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

torch.Size([5134, 128]) torch.Size([5134, 128]) torch.Size([5134, 128])


In [0]:
for name,param in model.named_parameters():
  param.requires_grad = False
model.cuda();

In [0]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [0]:
#Making predictions
class_preds = []
original_class_list = []

for batch in valid_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_segment, original_class = batch
  with torch.no_grad():
    outputs = model(b_input_ids, b_input_mask, b_segment)
    original_class_list.extend(list(original_class.detach().cpu().numpy()))
  classification_logits = outputs[0].detach().cpu().numpy()
  class_preds.extend(list(np.argmax(classification_logits,axis=1)))

In [60]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

print("F1 ",f1_score(original_class_list, class_preds))
print("Accuracy ",accuracy_score(original_class_list, class_preds))
print("Precision ",precision_score(original_class_list, class_preds))
print("Recall ",recall_score(original_class_list, class_preds))

F1  0.6391184573002755
Accuracy  0.9234514998052201
Precision  0.5658536585365853
Recall  0.7341772151898734


## Predicting from bert base, biober , scibert

In [63]:
bert_base = load_model('bert-base-uncased-task2-en.bin')
biobert = load_model('biobert-v1.1-pubmed-base-cased-en.bin')
scibert = load_model('scibert-scivocab-uncased-en.bin')

Loading model from bert-base-uncased-task2-en.bin...
Defaulting to linear classifier/regressor
Building sklearn text classifier...
Loading model from biobert-v1.1-pubmed-base-cased-en.bin...
Defaulting to linear classifier/regressor
Building sklearn text classifier...
Loading model from scibert-scivocab-uncased-en.bin...
Defaulting to linear classifier/regressor
Building sklearn text classifier...


In [64]:
bert_base_predictions = bert_base.predict(X_test)
biobert_predictions = biobert.predict(X_test)
scibert_predictions = scibert.predict(X_test)

In [66]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

print("F1 roberta large",f1_score(original_class_list, class_preds))
print("F1 bert base ",f1_score(y_test, bert_base_predictions))
print("F1 biobert ",f1_score(y_test, biobert_predictions))
print("F1 scibert ",f1_score(y_test, scibert_predictions))

F1 roberta large 0.6391184573002755
F1 bert base  0.6035751840168243
F1 biobert  0.5743801652892563
F1 scibert  0.582726326742976


In [188]:
101

array([1, 1, 1])

In [189]:
scibert_predictions[:3]

array([1, 1, 1])

In [0]:
from collections import Counter

ensemble_prediction = []
for i in range(len(y_test)):
  preds = [class_preds[i],
           bert_base_predictions[i],
           biobert_predictions[i], 
           scibert_predictions[i]]
  ensemble_prediction.append(Counter(preds).most_common(1)[0][0])

In [110]:
print("F1 ensemble ",f1_score(original_class_list, ensemble_prediction))
print("Accuracy ensemble ",accuracy_score(original_class_list, ensemble_prediction))
print("Precision ensemble ",precision_score(original_class_list, ensemble_prediction))
print("Recall ensemble ",recall_score(original_class_list, ensemble_prediction))

F1 ensemble  0.6489252814738997
Accuracy ensemble  0.9331904947409427
Precision ensemble  0.6302186878727635
Recall ensemble  0.6687763713080169


In [112]:
len(ensemble_prediction)

5134

In [113]:
test_predictions = pd.concat([test2.reset_index(drop=True), 
                              pd.DataFrame(ensemble_prediction, columns = ['pred'])], axis=1)
print(test_predictions.shape)
test_predictions.head(2)

(5134, 3)


,cleaned_tweet,tweet_id,pred
0,i do not fucking need humira,343909778008973312,0
1,"my retake is next friday , if i bloody fail ag...",352823276889837570,0


In [114]:
final_op_format = test_predictions[['tweet_id', 'pred']]
final_op_format.columns = ['tweet_id', 'Class']
print(final_op_format.shape)
final_op_format.head(2)

(5134, 2)


,tweet_id,Class
0,343909778008973312,0
1,352823276889837570,0


In [0]:
final_op_format.to_csv('./prediction_task2_en.tsv', sep='\t', index=False)

# Classifying tweets for task 3

In [117]:
tweets_to_classify = pd.read_csv('../classification_formatted_test_data.tsv', sep='\t')
print(tweets_to_classify.shape)
tweets_to_classify.head(2)

(428, 4)


,tweet_id,class,tweet,cleaned_tweet
0,332317478170546176,1,"do you have any medication allergies? ""asthma!...",do you have any medication allergies ? asthma ...
1,347806215776116737,1,"@ashleylvivian if #avelox has hurt your liver,...","if a velox has hurt your liver , avoid tylenol..."


In [0]:
tweets = tweets_to_classify['cleaned_tweet']
original_label = tweets_to_classify['class']

In [120]:
dev_input_ids, dev_attention_masks, dev_input_types = transformer_preprocess(list(tweets), tokenizer)
print(dev_input_ids.shape, dev_attention_masks.shape, dev_input_types.shape)

valid_data = TensorDataset(dev_input_ids, dev_attention_masks, 
                           dev_input_types)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

torch.Size([428, 128]) torch.Size([428, 128]) torch.Size([428, 128])


In [0]:
#Making predictions
class_preds = []

for batch in valid_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_segment = batch
  with torch.no_grad():
    outputs = model(b_input_ids, b_input_mask, b_segment)
  classification_logits = outputs[0].detach().cpu().numpy()
  class_preds.extend(list(np.argmax(classification_logits,axis=1)))

In [122]:
bert_base_predictions = bert_base.predict(tweets)
biobert_predictions = biobert.predict(tweets)
scibert_predictions = scibert.predict(tweets)

In [123]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

print("F1 roberta large",f1_score(list(original_label), class_preds))
print("F1 bert base ",f1_score(original_label, bert_base_predictions))
print("F1 biobert ",f1_score(original_label, biobert_predictions))
print("F1 scibert ",f1_score(original_label, scibert_predictions))

F1 roberta large 0.8662131519274376
F1 bert base  0.8252427184466019
F1 biobert  0.8175182481751825
F1 scibert  0.8221153846153847


In [0]:
from collections import Counter

ensemble_prediction = []
for i in range(len(original_label)):
  preds = [class_preds[i],
           bert_base_predictions[i],
           biobert_predictions[i], 
           scibert_predictions[i]]
  ensemble_prediction.append(Counter(preds).most_common(1)[0][0])

In [127]:
print("F1 ensemble ",f1_score(list(original_label), ensemble_prediction))
print("Accuracy ensemble ",accuracy_score(list(original_label), ensemble_prediction))
print("Precision ensemble ",precision_score(list(original_label), ensemble_prediction))
print("Recall ensemble ",recall_score(list(original_label), ensemble_prediction))

F1 ensemble  0.8463356973995271
Accuracy ensemble  0.8481308411214953
Precision ensemble  0.9421052631578948
Recall ensemble  0.7682403433476395


In [0]:
# WE WILL MAKE 2 PREDICTIONS. 1 WITH ROBERTA LARGE ONLY. OTHER IS AN ENSEMBLE

In [129]:
tweet_predictions = pd.concat([tweets_to_classify.reset_index(drop=True), 
                               pd.DataFrame(ensemble_prediction, columns = ['pred'])], axis=1)
print(tweet_predictions.shape)

#tweet_id tweet pred
adr_tweets = tweet_predictions[tweet_predictions['pred'] == 1]

(428, 5)


In [0]:
adr_tweets.to_csv('./adr_tweet_classification_op_ensemble_20200503.tsv', sep = '\t', index=False)

# Loading task3 tweets which has extracts from NER emsemble

In [139]:
span_op_df = pd.read_csv('../span_op_df_task3_ner.tsv')
print(span_op_df.shape)
span_op_df.head(2)

(119, 5)


,Unnamed: 0,tweet_id,start,end,extract
0,0,343944308828811264,8,19,hurt my foot
1,1,343944308828811264,66,79,tendon damage.


In [0]:
ratio_tweets_from_ner_which_are_non_adr_by_classifier = (len(set(span_op_df['tweet_id']) - set(adr_tweets['tweet_id'])))/\
                                                          len(set(span_op_df['tweet_id']))

In [0]:
adr_but_no_ner = list(set(adr_tweets['tweet_id']) - set(span_op_df['tweet_id']))
ner_but_no_adr = list(set(span_op_df['tweet_id']) - set(adr_tweets['tweet_id']))
ner_and_adr = list(set(span_op_df['tweet_id']) & set(adr_tweets['tweet_id']))

In [187]:
print("Total validation tweets = ", len(set(tweet_predictions['tweet_id'])))
print("ADR classified tweets = ", len(set(adr_tweets['tweet_id'])))
print("No ADR classified tweets = ", len(set(tweet_predictions['tweet_id'])) - len(set(adr_tweets['tweet_id'])))
print("Total NER tweets = ", len(set(span_op_df['tweet_id'])))
print("Total NER & ADR tweets = ", len(set(ner_and_adr)))
print("Total NER & No ADR tweets = ", len(set(ner_but_no_adr)))

Total validation tweets =  428
ADR classified tweets =  190
No ADR classified tweets =  238
Total NER tweets =  98
Total NER & ADR tweets =  54
Total NER & No ADR tweets =  44


## IDEA:


1.   Assume ADR classifier is 100% correct
2.   Create subset NER & ADR tweets
3.   For ADR tweets which don't have NER tags, perform QnA
4.   Combine (No ADR) from classifer & (ADR + NER) & (ADR + QnA - NER)



# Task3 QnA results

In [0]:
import json
import re
qna_json = json.load(open('../predictions_29march.json', 'r'))

In [156]:
test_data = pd.read_csv('../task3_validation.tsv', sep = '\t')
test_data = test_data.fillna('')
print(test_data.shape, len(qna_json))
tweet_id_list = list(test_data['tweet_id'])

(560, 9) 560


In [0]:
test_data_tweet_id_tweet_dict = {}
for ix,row in test_data.iterrows():
    test_data_tweet_id_tweet_dict[row['tweet_id']] = row['tweet']

In [0]:
def get_spans(query, doc):
    query = query.split(" ")
    doc = doc.split(" ")
    vec = [0]*len(doc)
    for q in query:
        for ix, d in enumerate(doc):
            if q == d:
                vec[ix] = 1
                break
    doc_len = []       
    for i in doc:
        doc_len.append(len(i))
    first_pos = 0
    last_pos = 0

    for ix, i in enumerate(vec):
        if i==1:
            first_pos = ix
            break

    for ix, i in reversed(list(enumerate(vec))):
        if i==1:
            last_pos = ix
            break

    start = sum(doc_len[:first_pos])+len(doc_len[:first_pos])
    end = start + sum(doc_len[first_pos:last_pos+1])+len(doc_len[first_pos:last_pos])
    return start,end


In [160]:
c = 0
op_list = []
exception_list = []
for k,v in qna_json.items():
    try:
        span = re.search(v.replace(')','\)').replace('(','\(').replace('"','\"').replace("'","\'").replace("?","\?"), 
                         test_data_tweet_id_tweet_dict[tweet_id_list[c]])
        op_list.append([tweet_id_list[c], v, span.start(), span.end()])
    except:
        st, en = get_spans(v, test_data_tweet_id_tweet_dict[tweet_id_list[c]])
        op_list.append([tweet_id_list[c], v, st, en])
        exception_list.append([tweet_id_list[c], v, st, en])
        print(v+" || "+test_data_tweet_id_tweet_dict[tweet_id_list[c]])
        print("\n")
    c += 1

answers back with, "uh... levaquin." #fml #pharmacyproblems || 5th year intern doesn't know the brand name for ciprofloxacin. answers back with, "uh... levaquin."  #fml #pharmacyproblems


kick in. going shopping || waiting on the trazodone to kick in.  going shopping with amanda kaye &amp; the fam tomorrow!  xd




In [164]:
qna_df = pd.DataFrame(op_list,columns = ['tweet_id', 'extract', 'begin', 'end'])
qna_df.to_csv('./qna_formatted_answers.tsv', sep = '\t', index = False)
print(qna_df.shape)
qna_df.head(2)

(560, 4)


,tweet_id,extract,begin,end
0,332317478170546176,allergies,27,36
1,347806215776116737,hurt your liver,30,45


# Combine QnA results and NER

In [205]:
adr_but_no_ner_qna = qna_df.merge(pd.DataFrame(adr_but_no_ner, columns = ['tweet_id']),
                                  how='inner',
                                  on='tweet_id')
adr_but_no_ner_qna = adr_but_no_ner_qna.drop_duplicates()
print(adr_but_no_ner_qna.shape)
adr_but_no_ner_qna['type'] = 'ADR'
adr_but_no_ner_qna.columns = ['tweet_id', 'extract', 'start', 'end', 'type']
adr_but_no_ner_qna.head(2)

(136, 4)


,tweet_id,extract,start,end,type
0,332317478170546176,allergies,27,36,ADR
1,347806215776116737,hurt your liver,30,45,ADR


In [201]:
adr_and_ner_df = span_op_df[['tweet_id', 'start', 
                             'end', 'extract']].merge(pd.DataFrame(ner_and_adr, columns = ['tweet_id']),
                                                      how='inner',
                                                      on='tweet_id')
adr_and_ner_df = adr_and_ner_df.drop_duplicates()
print(adr_and_ner_df.shape)
adr_and_ner_df['type'] = 'ADR'
adr_and_ner_df.head(2)

(65, 4)


,tweet_id,start,end,extract,type
0,343944308828811264,8,19,hurt my foot,ADR
1,343944308828811264,66,79,tendon damage.,ADR


In [202]:
no_adr_df = tweet_predictions[tweet_predictions['pred'] == 0][['tweet_id']].drop_duplicates()
no_adr_df['start'] = ''
no_adr_df['end'] = ''
no_adr_df['extract'] = ''
no_adr_df['type'] = ''
print(no_adr_df.shape)
no_adr_df.head(2)

(238, 5)


,tweet_id,start,end,extract,type
3,332540699692130304,,,,
4,332542581760204800,,,,


In [0]:
task3_op = pd.concat([adr_but_no_ner_qna[['tweet_id',	'start',	'end',	'extract',	'type']],
                      adr_and_ner_df[['tweet_id',	'start',	'end',	'extract',	'type']], 
                      no_adr_df[['tweet_id',	'start',	'end',	'extract',	'type']]])

In [209]:
print(task3_op.shape)
task3_op.head()

(439, 5)


,tweet_id,start,end,extract,type
0,332317478170546176,27,36,allergies,ADR
1,347806215776116737,30,45,hurt your liver,ADR
2,350336129817509888,75,114,length of focus today: about 30 seconds,ADR
4,348644040444637184,26,32,dreams,ADR
5,352217412046823425,84,98,tendon rupture,ADR


In [214]:
#Enriching output with tweet text for meddra mapper
task3_op = task3_op.merge(test_data[['tweet_id', 'tweet']].drop_duplicates(),
                          how='inner', on='tweet_id')
print(task3_op.shape)
task3_op.head()

(439, 6)


,tweet_id,start,end,extract,type,tweet
0,332317478170546176,27,36,allergies,ADR,"do you have any medication allergies? ""asthma!..."
1,347806215776116737,30,45,hurt your liver,ADR,"@ashleylvivian if #avelox has hurt your liver,..."
2,350336129817509888,75,114,length of focus today: about 30 seconds,ADR,"apparently, baclofen greatly exacerbates the ""..."
3,348644040444637184,26,32,dreams,ADR,@bilgeebiri oh robitussin dreams are notorious...
4,352217412046823425,84,98,tendon rupture,ADR,#eds friends! anybody taken #cipro? (antibioti...


In [215]:
#final sanity testing
test_tweet_ids = set(test_data['tweet_id'])
output_tweet_ids = set(task3_op['tweet_id'])
print(len(test_tweet_ids), len(output_tweet_ids), len(output_tweet_ids & test_tweet_ids))

428 428 428


In [0]:
task3_op.to_csv('./task3_op_before_meddra_mapping.tsv', sep='\t', index = False)